# Example application: computing the tree canopy for the Foreset Preserves of Cook County

In this example, we will compute a binary tree canopy map for [Theodore Stone Forest](https://fpdcc.com/places/locations/theodore-stone-forest/) and [Arie Crown Forest](https://fpdcc.com/places/locations/arie-crown-forest/).

In [ ]:
%cd ..
import os
from os import path

import detectree as dtr
import geopandas as gpd
import rasterio as rio
from rasterio import merge

from detectree_example import make_response_tiles, plot_utils, settings

Let us now use the Makefile at the root of this repository ensure that we have the required data inputs - i.e., the required tiles from Zurich's RGB orthophoto of summer 2014/15 that lie within Zurich's municipal boundaries - have been downloaded (see the background notebook for more details).

In [ ]:
!make aussersihl_tiles

## Train/test split

The `make` target above will store the required tiles in the 'data/interim/aussersihl_tiles' directory, which we will pass to the [`TrainingSelector`](https://detectree.readthedocs.io/en/latest/train_test_split.html#detectree.TrainingSelector.__init__) initialization method. For this example, we will use the ['cluster-I'](https://github.com/martibosch/detectree-example/blob/master/notebooks/cluster-I.ipynb) method (see Yang et al. [1]) to select the tiles that will be used to train the tree pixel classifier.

In [ ]:
ts = dtr.TrainingSelector(img_dir='data/interim/aussersihl_tiles')
split_df = ts.train_test_split(method='cluster-I')

## Interlude: computing the responses from LIDAR data

Since detectree uses a supervised learning approach, we need to manually provide the responses (i.e., binary images representing the ground-truth tree/non-tree masks) for the training tiles so that the tree/non-tree pixel classifier can be trained. 

There are many ways to obtain such masks, the most straight-forward being manual edition in a raster graphics editor software such as [GIMP](https://www.gimp.org/). In the case of this example dataset, the ground-truth masks can be extracted from [Zurich's 2014 LIDAR dataset](https://www.geolion.zh.ch/geodatensatz/show?gdsid=343). Such task is accomplished in the two cells below, whose content is out of the scope of detectree.

In [ ]:
!make download_lidar_shp

In [ ]:
lidar_gdf = gpd.read_file('data/raw/lidar/lidar2014.shp')

response_dir = 'data/interim/aussersihl_response-tiles'
if not path.exists(response_dir):
    os.mkdir(response_dir)

response_tiles = make_response_tiles.make_response_tiles(
    split_df, lidar_gdf, 'data/raw/lidar', response_dir)

## Training the classifier

We can now proceed to the training of the classifier. In detectree, this can be done with the `train_classifier` method of the `ClassifierTrainer` class, which accepts the train/test split data frame as the `split_df` keyword argument (see [its documentation](https://detectree.readthedocs.io/en/latest/pixel_classification.html#detectree.ClassifierTrainer.train_classifier) for more details). Note that the training of the classifier can take some time.

In [ ]:
clf = dtr.ClassifierTrainer().train_classifier(split_df=split_df,
                                               response_img_dir=response_dir)

## Predicting tree/non-tree labels

Once the classifier has been trained, it can be used to predict the tree/non-tree labels from any given tile. We will first create a directory where the predicted tiles will be dumped:

In [ ]:
output_dir = 'data/interim/aussersihl_pred-tiles'
if not path.exists(output_dir):
    os.mkdir(output_dir)

We can use the `classify_imgs` method of the `Classifier` class, which will predict the tree/non-tree labels for all the testing tiles of `split_df` (see [its documentation](https://detectree.readthedocs.io/en/latest/pixel_classification.html#detectree.Classifier.classify_imgs) for more details).

In [ ]:
c = dtr.Classifier()

pred_tiles = c.classify_imgs(split_df, output_dir=output_dir, clf=clf)

In [ ]:
pred_tiles

## Saving the tree canopy map as a single GeoTIFF file

Once we have tree pixels of all tiles, we can use the [`merge` function](https://rasterio.readthedocs.io/en/latest/api/rasterio.merge.html?highlight=merge#rasterio.merge.merge) of rasterio to assemble all the tiles into a single file, i.e., the binary tree canopy map.

We will first get a list of all the tree/non-tree tiles, i.e., the ones used for training (`response_tiles`) and the ones used for testing (`pred_tiles`), then read them with rasterio and pass them to the `merge` function:

In [ ]:
canopy_tiles = pred_tiles + response_tiles

canopy_arr, canopy_transform = merge.merge(
    [rio.open(canopy_tile) for canopy_tile in canopy_tiles])

We can now plot the result over a basemap (provided by means of the [contextily](https://github.com/geopandas/contextily) library):

In [ ]:
plot_utils.plot_canopy(canopy_arr, canopy_transform, figsize=(12, 8))

Finally, we can use rasterio to dump the tree canopy array to a GeoTIFF file:

In [ ]:
output_dir = 'data/processed'
if not path.exists(output_dir):
    os.mkdir(output_dir)

output_canopy_filepath = path.join(output_dir, 'aussersihl_canopy.tif')
output_dtype = rio.uint8

with rio.open(output_canopy_filepath,
              'w',
              driver='GTiff',
              width=canopy_arr.shape[2],
              height=canopy_arr.shape[1],
              count=1,
              crs=settings.CRS,
              transform=canopy_transform,
              dtype=output_dtype) as dst:
    dst.write(canopy_arr.astype(output_dtype))

## References

1. Yang, L., Wu, X., Praun, E., & Ma, X. (2009). Tree detection from aerial imagery. In Proceedings of the 17th ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems (pp. 131-137). ACM.